Bytes are writen as strings with a "b" in front of them.

They can be either be written as hex literals or the associated ASCIIvalues

See here: https://www.rapidtables.com/convert/number/hex-to-ascii.html

In [13]:
hex_literal = b"\x31\x30"
other_byte_literal = b"10"

print(hex_literal == other_byte_literal)
print(hex_literal)
print(other_byte_literal)

True
b'10'
b'10'


In [12]:
# numbers to and from bytes

# numbers aren't interpreted as their ASCII values - they're interpreted as
# their actual decimal value

small_byte_number = b"\x31"

# you need specify the endienness of the bytes - big endien is largest byte
# on the left, little endien is largest byte on the right
# see here: https://www.digital-detective.net/understanding-big-and-little-endian-byte-order/
# 0x31 == (3*16^1) + (1*16^0) == 49
print(int.from_bytes(small_byte_number, "big"))

bigger_byte_number = b"\x01\x23"

# 0x0123 = (0*16^3)+(1*16^2)+(2*16^1)+(3*16^0) in big endian
print(int.from_bytes(bigger_byte_number, "big"))

# 0x0123 = (0*16^1)+(1*16^0)+(2*16^3)+(3*16^2) in big endian
print(int.from_bytes(bigger_byte_number, "little"))


49
291
8961


`to_bytes` works in a similar manner to `from_bytes`.

You specify the int that you want to convert, the number of bytes you want to represent it, and its endianness.

In [49]:
# Representing 1024 using 6 bytes
val = int.to_bytes(1024, 6, "big")
print(val)


b'\x00\x00\x00\x00\x04\x00'


Quick note - data representation.

We covered above the certain values translate to ACII characters. The hex value `0x32` which is equal to `50` in decimal (`3*16^1 + 2*16^0`) translates to the ASCII character `2`. 

Below you'll see the int `50` converted to `b"\x002"`, which is strange, since the hex value that is equivalent to the decimal value `50` is `0x32`.

This is just shorthand. If you see two number preceded by `\x` then it is a hex literal. If you see a number or letter that doesn't belong to a hex pair `\xNN` then it is the ASCII representation of the byte. Don't let this confuse you. We can see below that they're the same thing.

In [48]:
v = int.to_bytes(50, 2, "big")
print(v)

z = int.from_bytes(b"\x00\x32", "big")
print(z)

print(b"\x002" == b"\x00\x32")

b'\x002'
50
True


307
259


True

In [ ]:
# Unpacking bytes
# See here: https://www.delftstack.com/howto/python/how-to-convert-bytes-to-integers/
